## 1.Load library

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
import matplotlib.pyplot as plt
import tensorflow as tf

## 2.Read Data

In [ ]:
tr = pd.read_csv('../input/tabular-playground-series-may-2021/train.csv')
tr.head()

In [ ]:
tr_X = tr.iloc[:,1:-1]
tr_y = tr.iloc[:,-1]

In [ ]:
le = LabelEncoder()
labels = le.fit_transform(tr_y)

In [ ]:
ms = MinMaxScaler()
ms.fit(tr_X)
tr_X_ms = ms.transform(tr_X)

## 3. Data reshape - Image array

In [ ]:
image_rows = 5
image_cols = 10
tr_X_ms_reshape = tr_X_ms.reshape(tr_X_ms.shape[0],image_rows,image_cols,1)

In [ ]:
images = np.array(tr_X_ms_reshape, dtype = 'float32')

In [ ]:
plt.imshow(images[0], cmap=plt.cm.binary)

In [ ]:
input_shapes = (image_rows, image_cols, 1)

## 4. Make CNN model

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(256, (2, 2), padding='same', activation = 'relu', input_shape = input_shapes), 
    tf.keras.layers.Conv2D(128, (2, 2), padding='same', activation = 'relu'),
    tf.keras.layers.Conv2D(64, (2, 2), padding='same', activation = 'relu'),
    tf.keras.layers.Conv2D(32, (2, 2), padding='same', activation = 'relu'),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation=tf.nn.relu),
    tf.keras.layers.Dense(64, activation=tf.nn.relu),
    tf.keras.layers.Dense(32, activation=tf.nn.relu),
    tf.keras.layers.Dense(4, activation=tf.nn.softmax)
])

model.compile(optimizer = 'RMSprop', loss = 'sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
def create_callbacks():
    
    cpk_path = './best_model.h5'
    
    checkpoint = tf.keras.callbacks.ModelCheckpoint(
        filepath=cpk_path,
        monitor='val_accuracy',
        mode='max',
        save_best_only=True,
        verbose=1,
    )

    reducelr = tf.keras.callbacks.ReduceLROnPlateau(
        monitor='val_accuracy',
        mode='max',
        factor=0.1,
        patience=3,
        verbose=0
    )

    earlystop = tf.keras.callbacks.EarlyStopping(
        monitor='val_accuracy',
        mode='max',
        patience=10, 
        verbose=1
    )
    
    callbacks = [checkpoint, reducelr, earlystop]         
    
    return callbacks

In [ ]:
def plot_hist(hist):
    plt.plot(hist.history["accuracy"])
    plt.plot(hist.history["val_accuracy"])
    plt.title("model accuracy")
    plt.ylabel("accuracy")
    plt.xlabel("epoch")
    plt.legend(["train", "validation"], loc="best")
    plt.show()

In [ ]:
Epochs = 100
Batch = 32

history = model.fit(images, labels, batch_size=Batch, epochs=Epochs, callbacks=create_callbacks(),validation_split=0.3)

In [ ]:
plot_hist(history)

## 5.Prediction

In [ ]:
te = pd.read_csv('../input/tabular-playground-series-may-2021/test.csv')
te.head()

In [ ]:
te_X = te.iloc[:,1:]
te_id = te.iloc[:,0]

In [ ]:
te_X_ms = ms.transform(te_X)
te_X_ms_reshape = te_X_ms.reshape(te_X_ms.shape[0],image_rows,image_cols,1)

In [ ]:
te_images = np.array(te_X_ms_reshape, dtype = 'float32')

In [ ]:
preds = model.predict_proba(te_images)

In [ ]:
preds

In [ ]:
submission = pd.DataFrame(preds, columns = ["Class_1", "Class_2", "Class_3", "Class_4"], index=te_id)

In [ ]:
submission.to_csv('./submission.csv')